
The R0 is between 2 and 6, meaning each infected person infects 2–6 others, which can cause the number of infected to double every few days.

In [ ]:

# https://informationisbeautiful.net/visualizations/covid-19-coronavirus-infographic-datapack/
# https://docs.google.com/spreadsheets/d/1g_YxmDfQx7aOU2DKzNZo9b-NTk62Bju6X3z6OuCa6gw/edit#gid=515684451

In [ ]:

command_str = '{sys.executable} -m pip install --upgrade statsmodels'.format(sys=sys)
print(command_str)
!{command_str}

In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
#pandemic_df = s.load_object('pandemic_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

%pprint
dir()

D:\Documents\Repositories\notebooks\StatsByCountry\ipynb\Corona Virus DataFrame Creation.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
Pretty printing has been turned OFF


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'nx', 'os', 'pd', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_st

In [3]:

asian_list = ['south_korea_df', 'hong_kong_df', 'japan_df', 'thailand_df', 'malaysia_df', 'china_df', 'vietnam_df',
              'singapore_df', 'philippines_df']
cumulative_list = ['china_df', 'czech_df', 'germany_df', 'hong_kong_df', 'iran_df', 'italy_df', 'japan_df', 'netherlands_df',
                   'singapore_df', 'slovakia_df', 'south_korea_df', 'spain_df', 'uk_df', 'usa_df']
for df_name in set(asian_list + cumulative_list):
    df = s.load_object(df_name)
    df.columns = [re.sub('Recoveries_Cumulative', 'Recovered_Cumulative', cn) for cn in df.columns]
    kwargs = {df_name: df}
    s.store_objects(**kwargs)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\slovakia_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\iran_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\usa_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\italy_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\germany_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\thailand_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\hong_kong_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\czech_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\singapore_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\china_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\save

In [ ]:

!start %windir%\explorer.exe "{os.path.abspath(os.path.dirname(notebook_path))}"

In [25]:

countries_list = ['Thailand', 'China', 'Japan', 'South Korea', 'Laos', 'Cambodia', 'Singapore', 'the Philippines', 'Hong Kong',
                  'Vietnam', 'Malaysia']
df_set = set(['{}_df'.format('_'.join(country.lower().split(' '))) for country in countries_list])

In [26]:

df_set

{'south_korea_df', 'hong_kong_df', 'japan_df', 'the_philippines_df', 'singapore_df', 'malaysia_df', 'china_df', 'vietnam_df', 'thailand_df', 'laos_df', 'cambodia_df'}

In [78]:

created_set = set()
df_name_list = []
for file_name in os.listdir(s.saves_pickle_folder):
    if file_name.endswith('_df.pickle'):
        df_name = file_name.split('.')[0]
        df_name_list.append(df_name)
        if df_name in df_set:
            created_set.add(df_name)
created_set

{'south_korea_df', 'hong_kong_df', 'japan_df', 'thailand_df', 'malaysia_df', 'china_df', 'vietnam_df', 'singapore_df'}

In [ ]:

for df_name in list(df_set - created_set):
    country_name = df_name[:-3].title()
    country_name = re.sub('The_', 'the_', country_name)
    rm_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_{}'.format(country_name)
    !"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}


---
# Create the DataFrame

In [84]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
tables_list = get_page_tables(tables_url)

[(0, (242, 6)), (1, (12, 2))]


In [94]:

country_populations_df = tables_list[0].copy()
country_populations_df.columns = ['Rank', 'Country', 'Population', 'world_population_percent', 'Date', 'Source']
country_populations_df = country_populations_df.loc[:240]
for column_name in country_populations_df.columns:
    if column_name == 'Country':
        country_populations_df[column_name] = country_populations_df[column_name].map(lambda x: str(x).split('[')[0])
    elif column_name in ['Population', 'Rank']:
        country_populations_df[column_name] = country_populations_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        country_populations_df[column_name] = pd.to_numeric(country_populations_df[column_name], errors='coerce')
country_populations_df.set_index('Country', drop=True, inplace=True)
columns_list = ['Rank', 'Population', 'Date', 'Source']
country_populations_df = country_populations_df[columns_list]
s.store_objects(country_populations_df=country_populations_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\country_populations_df.pickle


In [80]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_States_medical_cases'
tables_list = get_page_tables(tables_url)

[(0, (43, 61)), (1, (30, 56)), (2, (66, 2)), (4, (22, 2)), (5, (21, 2)), (14, (11, 2)), (15, (10, 2)), (17, (8, 2)), (18, (7, 2)), (6, (5, 2)), (7, (5, 2)), (16, (5, 2)), (10, (4, 2)), (8, (3, 2)), (11, (3, 2)), (12, (3, 2)), (13, (2, 2)), (3, (1, 2)), (9, (1, 2))]


In [83]:

usa_df = tables_list[0].copy()
usa_df.columns = ['Date', 'West_AK', 'West_AZ', 'West_CA', 'West_CO', 'West_HI', 'West_ID', 'West_MT', 'West_NM', 'West_NV', 'West_OR',
                  'West_UT', 'West_WA', 'West_WY', 'Midwest_IA', 'Midwest_IL', 'Midwest_IN', 'Midwest_KS', 'Midwest_MI', 'Midwest_MN',
                  'Midwest_MO', 'Midwest_ND', 'Midwest_NE', 'Midwest_OH', 'Midwest_OK', 'Midwest_SD', 'Midwest_WI', 'South_AL',
                  'South_AR', 'South_FL', 'South_GA', 'South_KY', 'South_LA', 'South_MS', 'South_NC', 'South_SC', 'South_TN', 'South_TX',
                  'South_VA', 'South_WV', 'Northeast_CT', 'Northeast_DC', 'Northeast_DE', 'Northeast_MA', 'Northeast_MD',
                  'Northeast_ME', 'Northeast_NH', 'Northeast_NJ', 'Northeast_NY', 'Northeast_PA', 'Northeast_RI', 'Northeast_VT',
                  'Territories_GU', 'Territories_PR', 'Territories_VI', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New',
                  'Deaths_Cumulative', 'Recovered_New', 'Recovered_Cumulative']
usa_df = usa_df.loc[:37]
date_format = '%b %d, %Y'
for column_name in usa_df.columns:
    if column_name == 'Date':
        usa_df[column_name] = usa_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        usa_df[column_name] = pd.to_datetime(usa_df[column_name])
        pass
    else:
        usa_df[column_name] = usa_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        usa_df[column_name] = pd.to_numeric(usa_df[column_name], errors='coerce')
s.store_objects(usa_df=usa_df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
#usa_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\usa_df.pickle


In [60]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_the_Philippines'
tables_list = get_page_tables(tables_url)

[(2, (636, 10)), (10, (69, 7)), (6, (29, 5)), (12, (65, 2)), (5, (13, 8)), (3, (25, 4)), (4, (15, 4)), (14, (22, 2)), (15, (21, 2)), (8, (13, 3)), (29, (12, 3)), (1, (12, 2)), (24, (10, 2)), (25, (9, 2)), (27, (8, 2)), (28, (7, 2)), (7, (5, 2)), (16, (5, 2)), (17, (5, 2)), (30, (5, 2)), (20, (4, 2)), (26, (4, 2)), (18, (3, 2)), (21, (3, 2)), (22, (3, 2)), (23, (2, 2)), (0, (1, 2)), (9, (1, 2)), (11, (1, 2)), (13, (1, 2)), (19, (1, 2))]


In [69]:

philippines_df = tables_list[6].copy()
philippines_df.columns = philippines_df.iloc[0].tolist()
columns_list = [1, 3]
philippines_df = philippines_df.iloc[:, columns_list]
philippines_df.columns = ['Date', 'Confirmed_New']
#philippines_df.Date.to_dict()
idx_list = [1] + list(range(3, 6)) + [7] + list(range(9, 28))
match_series = philippines_df.index.isin(idx_list)
philippines_df = philippines_df[match_series]
for column_name in philippines_df.columns:
    if column_name == 'Date':
        philippines_df[column_name] = pd.to_datetime(philippines_df[column_name])
        pass
    else:
        philippines_df[column_name] = philippines_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        philippines_df[column_name] = pd.to_numeric(philippines_df[column_name], errors='coerce')
s.store_objects(philippines_df=philippines_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\philippines_df.pickle


In [52]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Thailand'
tables_list = get_page_tables(tables_url)

[(5, (342, 12)), (3, (65, 5)), (6, (19, 7)), (7, (65, 2)), (9, (22, 2)), (10, (21, 2)), (1, (13, 2)), (19, (10, 2)), (20, (9, 2)), (22, (8, 2)), (23, (7, 2)), (4, (2, 6)), (11, (5, 2)), (12, (5, 2)), (15, (4, 2)), (21, (4, 2)), (13, (3, 2)), (16, (3, 2)), (17, (3, 2)), (18, (2, 2)), (0, (1, 2)), (2, (1, 2)), (8, (1, 2)), (14, (1, 2))]


In [59]:

thailand_df = tables_list[3].copy()
thailand_df.columns = thailand_df.iloc[0].tolist()
columns_list = [1, 3]
thailand_df = thailand_df.iloc[:, columns_list]
thailand_df.columns = ['Date', 'Confirmed_New']
#thailand_df.Date.to_dict()
idx_list = [1, 3,] + list(range(5, 20)) + list(range(21, 26)) + list(range(27, 29)) + list(range(30, 42)) + list(range(43, 64))
match_series = thailand_df.index.isin(idx_list)
thailand_df = thailand_df[match_series]
for column_name in thailand_df.columns:
    if column_name == 'Date':
        thailand_df[column_name] = pd.to_datetime(thailand_df[column_name])
        pass
    else:
        thailand_df[column_name] = thailand_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        thailand_df[column_name] = pd.to_numeric(thailand_df[column_name], errors='coerce')
s.store_objects(thailand_df=thailand_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\thailand_df.pickle


In [41]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Vietnam'
tables_list = get_page_tables(tables_url)

[(4, (121, 12)), (2, (47, 5)), (5, (65, 2)), (3, (21, 3)), (7, (22, 2)), (8, (21, 2)), (1, (11, 2)), (17, (10, 2)), (18, (9, 2)), (20, (8, 2)), (21, (7, 2)), (9, (5, 2)), (10, (5, 2)), (13, (4, 2)), (19, (4, 2)), (11, (3, 2)), (14, (3, 2)), (15, (3, 2)), (16, (2, 2)), (0, (1, 2)), (6, (1, 2)), (12, (1, 2))]


In [51]:

vietnam_df = tables_list[2].copy()
vietnam_df.columns = vietnam_df.iloc[0].tolist()
columns_list = [1, 3]
vietnam_df = vietnam_df.iloc[:, columns_list]
vietnam_df.columns = ['Date', 'Confirmed_New']
#vietnam_df.Date.to_dict()
idx_list = [1, 3,] + list(range(5, 19)) + list(range(20, 23)) + [24] + list(range(26, 46))
match_series = vietnam_df.index.isin(idx_list)
vietnam_df = vietnam_df[match_series]
for column_name in vietnam_df.columns:
    if column_name == 'Date':
        vietnam_df[column_name] = pd.to_datetime(vietnam_df[column_name])
        pass
    else:
        vietnam_df[column_name] = vietnam_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        vietnam_df[column_name] = pd.to_numeric(vietnam_df[column_name], errors='coerce')
s.store_objects(vietnam_df=vietnam_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\vietnam_df.pickle


In [29]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Malaysia'
tables_list = get_page_tables(tables_url)

[(8, (120, 8)), (6, (113, 5)), (7, (46, 7)), (3, (56, 5)), (5, (13, 18)), (9, (20, 9)), (12, (65, 2)), (14, (22, 2)), (15, (21, 2)), (2, (16, 2)), (4, (6, 5)), (24, (10, 2)), (25, (9, 2)), (11, (4, 4)), (27, (8, 2)), (10, (3, 5)), (28, (7, 2)), (16, (5, 2)), (17, (5, 2)), (20, (4, 2)), (26, (4, 2)), (18, (3, 2)), (21, (3, 2)), (22, (3, 2)), (23, (2, 2)), (0, (1, 2)), (1, (1, 2)), (13, (1, 2)), (19, (1, 2))]


In [40]:

from datetime import datetime

malaysia_df = tables_list[7].copy()
malaysia_df.columns = ['Date', 'Selangor', 'Johor', 'Kuala_Lumpur', 'Kedah', 'North_Sembilan', 'Confirmed_New']
malaysia_df = malaysia_df.loc[:43]
date_format = '%d %B, %Y'
for column_name in malaysia_df.columns:
    if column_name == 'Date':
        malaysia_df[column_name] = malaysia_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        malaysia_df[column_name] = pd.to_datetime(malaysia_df[column_name])
        pass
    else:
        malaysia_df[column_name] = malaysia_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        malaysia_df[column_name] = pd.to_numeric(malaysia_df[column_name], errors='coerce')
s.store_objects(malaysia_df=malaysia_df)
#columns_list = ['Date', 'Confirmed_New']
#malaysia_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\malaysia_df.pickle


In [15]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_Kingdom_medical_cases'
tables_list = get_page_tables(tables_url)

[(0, (34, 19)), (1, (65, 2)), (3, (22, 2)), (4, (21, 2)), (13, (10, 2)), (14, (9, 2)), (16, (8, 2)), (17, (7, 2)), (5, (5, 2)), (6, (5, 2)), (9, (4, 2)), (15, (4, 2)), (7, (3, 2)), (10, (3, 2)), (11, (3, 2)), (12, (2, 2)), (2, (1, 2)), (8, (1, 2))]


In [21]:

uk_df = tables_list[0].copy()
uk_df.columns = ['Date', 'England_East', 'England_London', 'England_Midlands', 'England_NE_Yorks', 'England_North_West',
                 'England_South_East', 'England_South_West', 'England_Unclassified', 'Northern_Ireland', 'Scotland', 'Wales',
                 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Tested_New', 'Tested_Cumulative',
                 'Sources']
uk_df = uk_df.loc[:31]
#date_format = '%Y/%M/%d'
for column_name in uk_df.columns:
    if column_name == 'Date':
        #uk_df[column_name] = uk_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        uk_df[column_name] = pd.to_datetime(uk_df[column_name])
        pass
    else:
        uk_df[column_name] = uk_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        uk_df[column_name] = pd.to_numeric(uk_df[column_name], errors='coerce')
s.store_objects(uk_df=uk_df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
#uk_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\uk_df.pickle


In [7]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/Spain_medical_cases'
tables_list = get_page_tables(tables_url)

[(6, (37, 30)), (0, (36, 11)), (3, (29, 10)), (2, (12, 11)), (8, (62, 2)), (1, (20, 6)), (4, (12, 10)), (5, (12, 10)), (10, (19, 2)), (11, (18, 2)), (20, (10, 2)), (21, (9, 2)), (23, (8, 2)), (24, (7, 2)), (13, (5, 2)), (16, (4, 2)), (22, (4, 2)), (14, (3, 2)), (17, (3, 2)), (18, (3, 2)), (12, (2, 2)), (19, (2, 2)), (7, (1, 3)), (9, (1, 2)), (15, (1, 2))]


In [14]:

spain_df = tables_list[6].copy()
districts_list = ['AN', 'AR', 'AS', 'IB', 'CN', 'CB', 'CM', 'CL', 'CT', 'CE', 'VC', 'EX', 'GA', 'MD', 'ML', 'MU', 'NA',
                  'PV', 'RI']
summaries_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'ICU_Cumulative',
                  'Recovered_Cumulative', 'Tested_Cumulative']
spain_df.columns = ['Date', 'Time'] + districts_list + summaries_list + ['Refs', 'Notes']
spain_df = spain_df.loc[:30]
for column_name in spain_df.columns:
    if column_name == 'Date':
        spain_df[column_name] = pd.to_datetime(spain_df[column_name])
        pass
    elif column_name == 'Time':
        spain_df[column_name] = pd.to_timedelta(spain_df[column_name]+':00')
        pass
    else:
        spain_df[column_name] = spain_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        spain_df[column_name] = pd.to_numeric(spain_df[column_name], errors='coerce')
df_columns_list = districts_list + summaries_list
df = pd.DataFrame([], columns=df_columns_list)
summed_list = districts_list + ['Confirmed_New', 'Deaths_New']
for column_name in summed_list:
    new_group = spain_df.groupby('Date')[column_name]
    df[column_name] = new_group.agg(np.sum)
for column_name in ['Confirmed_Cumulative', 'Deaths_Cumulative', 'ICU_Cumulative', 'Recovered_Cumulative', 'Tested_Cumulative']:
    cumulative_group = spain_df.groupby('Date')[column_name]
    df[column_name] = cumulative_group.agg(max)
df.reset_index(level=0, inplace=True)
s.store_objects(spain_df=df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
#df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\spain_df.pickle


In [3]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_South_Korea'
tables_list = get_page_tables(tables_url)

[(8, (75, 28)), (2, (30, 13)), (3, (61, 5)), (5, (20, 7)), (7, (20, 7)), (9, (62, 2)), (4, (13, 7)), (6, (8, 5)), (11, (19, 2)), (12, (18, 2)), (1, (14, 2)), (21, (10, 2)), (22, (9, 2)), (24, (8, 2)), (25, (7, 2)), (14, (5, 2)), (17, (4, 2)), (23, (4, 2)), (15, (3, 2)), (18, (3, 2)), (19, (3, 2)), (13, (2, 2)), (20, (2, 2)), (0, (1, 2)), (10, (1, 2)), (16, (1, 2))]


In [6]:

import numpy as np

south_korea_df = tables_list[8].copy()
districts_list = ['Gyeonggi_Incheon', 'Gyeonggi_Seoul', 'Gyeonggi_Gyeonggi', 'Gangwon_Gangwon',
                  'Gyeongsang_Gyeongbuk', 'Gyeongsang_Daegu', 'Gyeongsang_Gyeongnam', 'Gyeongsang_Busan',
                  'Gyeongsang_Ulsan', 'Chungcheong_Chungbuk', 'Chungcheong_Sejong', 'Chungcheong_Daejeon',
                  'Chungcheong_Chungnam', 'Jeolla_Jeonbuk', 'Jeolla_Gwangju', 'Jeolla_Jeonnam',
                  'Jeolla_Jeju', 'Airport Screening']
summaries_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative',
                  'Tested_Cumulative', 'Tested_New', 'Discharged_Cumulative']
south_korea_df.columns = ['Date', 'Time'] + districts_list + summaries_list + ['Sources']
#south_korea_df.Date.to_dict()
idx_list = list(range(18)) + list(range(19, 42)) + list(range(43, 69))
match_series = south_korea_df.index.isin(idx_list)
south_korea_df = south_korea_df[match_series]
for column_name in south_korea_df.columns:
    if column_name == 'Date':
        south_korea_df[column_name] = pd.to_datetime(south_korea_df[column_name])
        pass
    elif column_name == 'Time':
        south_korea_df[column_name] = pd.to_timedelta(south_korea_df[column_name]+':00')
        pass
    else:
        south_korea_df[column_name] = south_korea_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        south_korea_df[column_name] = pd.to_numeric(south_korea_df[column_name], errors='coerce')
df_columns_list = districts_list + summaries_list
df = pd.DataFrame([], columns=df_columns_list)
summed_list = districts_list + ['Confirmed_New', 'Deaths_New', 'Tested_New']
for column_name in summed_list:
    new_group = south_korea_df.groupby('Date')[column_name]
    df[column_name] = new_group.agg(np.sum)
for column_name in ['Confirmed_Cumulative', 'Deaths_Cumulative', 'Tested_Cumulative', 'Discharged_Cumulative']:
    cumulative_group = south_korea_df.groupby('Date')[column_name]
    df[column_name] = cumulative_group.agg(max)
df.reset_index(level=0, inplace=True)
s.store_objects(south_korea_df=df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative']
#df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\south_korea_df.pickle


In [172]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Japan'
tables_list = get_page_tables(tables_url)

[(4, (62, 50)), (3, (55, 7)), (2, (70, 5)), (5, (62, 2)), (7, (19, 2)), (8, (18, 2)), (1, (10, 2)), (17, (10, 2)), (18, (9, 2)), (20, (8, 2)), (21, (7, 2)), (10, (5, 2)), (13, (4, 2)), (19, (4, 2)), (11, (3, 2)), (14, (3, 2)), (15, (3, 2)), (9, (2, 2)), (16, (2, 2)), (0, (1, 2)), (6, (1, 2)), (12, (1, 2))]


In [179]:

japan_df = tables_list[4].copy()
japan_df.columns = ['Date', 'Hokkaidō', 'Honshū_Tōhoku_Aomori', 'Honshū_Tōhoku_Akita',
                    'Honshū_Tōhoku_Miyagi', 'Honshū_Tōhoku_Fukushima', 'Honshū_Kantō_Tochigi', 'Honshū_Kantō_Ibaraki',
                    'Honshū_Kantō_Chiba', 'Honshū_Kantō_Tōkyō', 'Honshū_Kantō_Kanagawa', 'Honshū_Kantō_Saitama',
                    'Honshū_Kantō_Gunma', 'Honshū_Chūbu_Niigata', 'Honshū_Chūbu_Nagano', 'Honshū_Chūbu_Yamanashi',
                    'Honshū_Chūbu_Shizuoka', 'Honshū_Chūbu_Aichi', 'Honshū_Chūbu_Ishikawa', 'Honshū_Chūbu_Fukui',
                    'Honshū_Chūbu_Gifu', 'Honshū_Kansai_Mie', 'Honshū_Kansai_Shiga', 'Honshū_Kansai_Wakayama', 'Honshū_Kansai_Nara',
                    'Honshū_Kansai_Kyōto', 'Honshū_Kansai_Ōsaka', 'Honshū_Kansai_Hyōgo', 'Honshū_Chūgoku_Okayama',
                    'Honshū_Chūgoku_Hiroshima', 'Honshū_Chūgoku_Yamaguchi', 'Shikoku_Shikoku_Kagawa',
                    'Shikoku_Shikoku_Tokushima', 'Shikoku_Shikoku_Kōchi', 'Shikoku_Shikoku_Ehime', 'Kyūshū_Kyūshū_Ōita',
                    'Kyūshū_Kyūshū_Fukuoka', 'Kyūshū_Kyūshū_Saga', 'Kyūshū_Kyūshū_Nagasaki', 'Kyūshū_Kyūshū_Kumamoto',
                    'Kyūshū_Kyūshū_Miyazaki', 'Okinawa', 'Airport', 'Diamond_Princess_Workers', 'Repatriated_Total',
                    'Abroad_Total', 'Confirmed_New', 'Confirmed_Total', 'Tested_Total', 'Sources']
#japan_df.Date.to_dict()
idx_list = list(range(14)) + list(range(15, 32)) + list(range(33, 56))
match_series = japan_df.index.isin(idx_list)
japan_df = japan_df[match_series]
#date_format = '%Y/%M/%d'
for column_name in japan_df.columns:
    if column_name == 'Date':
        #japan_df[column_name] = japan_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        japan_df[column_name] = pd.to_datetime(japan_df[column_name])
        pass
    else:
        japan_df[column_name] = japan_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        japan_df[column_name] = pd.to_numeric(japan_df[column_name], errors='coerce')
s.store_objects(japan_df=japan_df)
#columns_list = ['Date', 'Confirmed_Total', 'Confirmed_New']
#japan_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\japan_df.pickle


In [159]:

#tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/Italy_medical_cases'
xlsx_dir = os.path.join(s.saves_folder, 'xlsx')
file_path = os.path.join(xlsx_dir, 'Daily_COVID-19_cases_in_Italy_by_region.xlsx')
italy_df = pd.read_excel(file_path)
italy_df.columns = ['Date', 'North_West_VDA', 'North_West_LIG', 'North_West_PIE', 'North_West_LOM', 'North_East_VEN',
                    'North_East_TN', 'North_East_BZ', 'North_East_FVG', 'North_East_EMR', 'Centre_MAR', 'Centre_TOS', 'Centre_UMB',
                    'Centre_LAZ', 'South_ABR', 'South_MOL', 'South_CAM', 'South_BAS', 'South_PUG', 'South_CAL', 'Islands_SIC',
                    'Islands_SAR', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Active_ICU', 'Active_Cumulative',
                    'Recovered_Cumulative', 'Tested_Cumulative', 'References', 'Notes']
italy_df = italy_df.iloc[1:36]
#date_format = '%Y/%M/%d'
for column_name in italy_df.columns:
    if column_name == 'Date':
        #italy_df[column_name] = italy_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        italy_df[column_name] = pd.to_datetime(italy_df[column_name])
        pass
    else:
        italy_df[column_name] = italy_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        italy_df[column_name] = pd.to_numeric(italy_df[column_name], errors='coerce')
s.store_objects(italy_df=italy_df)
s.save_dataframes(italy_df=italy_df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Active_ICU', 'Active_Cumulative']
#italy_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\italy_df.pickle
Saving to D:\Documents\Repositories\notebooks\StatsByCountry\saves\csv\italy_df.csv


In [134]:

page_tables_list = get_page_tables('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Iran')

[(6, (35, 37)), (4, (36, 6)), (3, (36, 5)), (8, (62, 2)), (10, (19, 2)), (11, (18, 2)), (1, (14, 2)), (20, (10, 2)), (21, (9, 2)), (23, (8, 2)), (24, (7, 2)), (13, (5, 2)), (16, (4, 2)), (22, (4, 2)), (14, (3, 2)), (17, (3, 2)), (18, (3, 2)), (5, (2, 2)), (12, (2, 2)), (19, (2, 2)), (0, (1, 2)), (2, (1, 2)), (7, (1, 2)), (9, (1, 2)), (15, (1, 2))]


In [141]:

iran_df = page_tables_list[6].copy()
iran_df.columns = ['Date', 'Region_1_Qom', 'Region_1_Teh', 'Region_1_Maz', 'Region_1_Alb', 'Region_1_Sem', 'Region_1_Gol',
                   'Region_1_Qaz', 'Region_2_Esf', 'Region_2_Frs', 'Region_2_Hor', 'Region_2_Koh', 'Region_2_Cha', 'Region_2_Bus',
                   'Region_3_Gil', 'Region_3_Ard', 'Region_3_Azs', 'Region_3_Azg', 'Region_3_Kur', 'Region_3_Zan', 'Region_4_Mar',
                   'Region_4_Ham', 'Region_4_Khz', 'Region_4_Krs', 'Region_4_Lor', 'Region_4_Ilm', 'Region_5_Khr', 'Region_5_Sis',
                   'Region_5_Yaz', 'Region_5_Khs', 'Region_5_Ker', 'Region_5_Khn', 'Confirmed_New', 'Confirmed_Total',
                   'Deaths_New', 'Deaths_Total', 'Sources']
iran_df = iran_df.loc[:32]
#date_format = '%Y/%M/%d'
for column_name in iran_df.columns:
    if column_name == 'Date':
        #iran_df[column_name] = iran_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        iran_df[column_name] = pd.to_datetime(iran_df[column_name])
        pass
    else:
        iran_df[column_name] = iran_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        iran_df[column_name] = pd.to_numeric(iran_df[column_name], errors='coerce')
s.store_objects(iran_df=iran_df)
#columns_list = ['Date', 'Confirmed_Total', 'Deaths_Total', 'Confirmed_New', 'Deaths_New']
#iran_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\iran_df.pickle


In [34]:

slovakia_df = s.load_object('slovakia_df')
slovakia_df.columns = ['Date', 'Bratislava', 'Žilina', 'Košice', 'Trnava', 'Trenčín', 'Prešov', 'Banská_Bystrica',
                       'Nitra', 'Confirmed_New', 'Confirmed_Total', 'Deaths_New', 'Deaths_Total',
                       'Recoveries_Total', 'Tested_Total', 'References']
slovakia_df = slovakia_df.loc[:17]
for column_name in slovakia_df.columns:
    if column_name == 'Date':
        slovakia_df[column_name] = pd.to_datetime(slovakia_df[column_name])
        pass
    else:
        slovakia_df[column_name] = slovakia_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        slovakia_df[column_name] = pd.to_numeric(slovakia_df[column_name], errors='coerce')
s.store_objects(slovakia_df=slovakia_df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Total', 'Deaths_New', 'Deaths_Total']
#slovakia_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\slovakia_df.pickle


In [28]:

singapore_df = s.load_object('singapore_df')
singapore_df.columns = singapore_df.iloc[0].tolist()
singapore_df = singapore_df.loc[1:61]
columns_list = [1, 3, 4]
singapore_df = singapore_df.iloc[:, columns_list]
singapore_df.columns = ['Date', 'Confirmed_New', 'Deaths_New']
idx_list = list(range(11)) + list(range(12, 62))
match_series = singapore_df.index.isin(idx_list)
singapore_df = singapore_df[match_series]
for column_name in singapore_df.columns:
    if column_name == 'Date':
        singapore_df[column_name] = pd.to_datetime(singapore_df[column_name])
        pass
    else:
        singapore_df[column_name] = singapore_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        singapore_df[column_name] = pd.to_numeric(singapore_df[column_name], errors='coerce')
s.store_objects(singapore_df=singapore_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\singapore_df.pickle


In [19]:

netherlands_df = s.load_object('netherlands_df')
netherlands_df.columns = netherlands_df.iloc[0].tolist()
netherlands_df = netherlands_df.loc[1:27]
columns_list = [1, 3, 4]
netherlands_df = netherlands_df.iloc[:, columns_list]
netherlands_df.columns = ['Date', 'Confirmed_New', 'Deaths_New']
for column_name in netherlands_df.columns:
    if column_name == 'Date':
        netherlands_df[column_name] = pd.to_datetime(netherlands_df[column_name])
        pass
    else:
        netherlands_df[column_name] = netherlands_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        netherlands_df[column_name] = pd.to_numeric(netherlands_df[column_name], errors='coerce')
s.store_objects(netherlands_df=netherlands_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\netherlands_df.pickle


In [ ]:

hong_kong_df = s.load_object('hong_kong_df')
hong_kong_df.columns = hong_kong_df.iloc[0].tolist()
hong_kong_df = hong_kong_df.loc[1:62]
columns_list = [1, 4]
hong_kong_df = hong_kong_df.iloc[:, columns_list]
hong_kong_df.columns = ['Date', 'Confirmed_New']
hong_kong_df.Confirmed_New = hong_kong_df.Confirmed_New.map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
hong_kong_df.Confirmed_New = pd.to_numeric(hong_kong_df.Confirmed_New, errors='coerce')
hong_kong_df.Date = pd.to_datetime(hong_kong_df.Date)
s.store_objects(hong_kong_df=hong_kong_df)

In [ ]:

germany_df = s.load_object('germany_df')
germany_df.columns = ['Date', 'Baden-Württemberg', 'Bavaria', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hesse',
                      'Mecklenburg-Vorpommern', 'Lower Saxony', 'North Rhine-Westphalia', 'Rhineland-Palatinate', 'Saarland',
                      'Saxony', 'Saxony-Anhalt', 'Schleswig-Holstein', 'Thuringia', 'Confirmed_Repatriated', 'Confirmed_Total',
                      'Deaths_Total', 'Confirmed_New', 'Deaths_New']
date_format = '%d %b, %Y'
for column_name in germany_df.columns:
    if column_name == 'Date':
        germany_df[column_name] = germany_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        germany_df[column_name] = pd.to_datetime(germany_df[column_name])
        pass
    else:
        germany_df[column_name] = germany_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        germany_df[column_name] = pd.to_numeric(germany_df[column_name], errors='coerce')
s.store_objects(germany_df=germany_df)
#columns_list = ['Date', 'Confirmed_Repatriated', 'Confirmed_Total', 'Deaths_Total', 'Confirmed_New', 'Deaths_New']
#germany_df[columns_list]

In [ ]:

czech_df = s.load_object('czech_df')
czech_df.columns = ['Date', 'Confirmed_New', 'Confirmed_Total', 'Recovered_New', 'Recovered_Total', 'Deaths_New', 'Deaths_Total',
                    'Tested_Total']
date_format = '%d %B, %Y'
for column_name in czech_df.columns:
    if column_name == 'Date':
        czech_df[column_name] = czech_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        czech_df[column_name] = pd.to_datetime(czech_df[column_name])
        pass
    else:
        czech_df[column_name] = czech_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        czech_df[column_name] = pd.to_numeric(czech_df[column_name], errors='coerce')
s.store_objects(czech_df=czech_df)

In [ ]:

china_df = s.load_object('china_df')
china_df.columns = china_df.iloc[0].tolist()
china_df = china_df.loc[1:77]
columns_list = [1, 4]
china_df = china_df.iloc[:, columns_list]
china_df.columns = ['Date', 'Confirmed_New']
china_df.Confirmed_New = china_df.Confirmed_New.map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
china_df.Confirmed_New = pd.to_numeric(china_df.Confirmed_New, errors='coerce')
china_df.Date = pd.to_datetime(china_df.Date)
s.store_objects(china_df=china_df)

In [ ]:

page_title = '2019–20 coronavirus pandemic'
page_obj = wikipedia.page(title=page_title)
print(['page_obj.{}'.format(fn) for fn in dir(page_obj) if 'link' in fn.lower()])

In [ ]:

print(['page_obj.{}'.format(fn) for fn in dir(page_obj) if 'url' in fn.lower()])

In [ ]:

countries_list = ['Italy']
locations_list = [link for link in set(page_obj.links) if ' in ' in link]
for page_title in locations_list:
    if any(map(lambda x: x in page_title, countries_list)):
        page_obj = wikipedia.page(title=page_title)
        tables_url = page_obj.url
        tables_list = get_page_tables(tables_url)
        break

In [ ]:

print(page_title)
tables_list[4]

In [ ]:

locations_list

In [ ]:

pandemic_df = s.load_object('pandemic_df')
pandemic_df.loc[12, 'R0_low'] = 2
pandemic_df.loc[12, 'R0_high'] = 6
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

pandemic_df.set_index('Disease', drop=True, inplace=True)
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

csv_name = 'COVID-19'
csv_path = os.path.abspath(os.path.join(s.data_csv_folder, '{}.csv'.format(csv_name)))
covid19_df = pd.read_csv(csv_path, encoding=s.encoding_type, header=[0, 1])
covid19_df

In [ ]:

name_regex = re.compile(r'[^0-9A-Za-z]+')
[name_regex.sub('_', '{} {}'.format(cn_tuple[0], cn_tuple[1]).lower()) for cn_tuple in covid19_df.columns]

In [ ]:

covid19_df.columns = ['Disease', 'deaths_per_day_global_rounded', 'total_news_mentions_millions_data_retrieved_3rd_mar_2020',
                      'news_mentions_per_death', 'CFR_low', 'R0_low', 'annual_global_fatalities_latest_data_year',
                      'new_cases_per_year_latest_data_year', 'days_of_outbreak', 'fatalty_notes', 'source', 'url']
covid19_df.set_index('Disease', drop=True, inplace=True)
s.store_objects(covid19_df=covid19_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
pandemic_df.index = ['Pertussis', 'Diphtheria', 'Measles', 'Smallpox', 'Rubella', 'Mumps', 'SARS', 'MERS', 'Swine flu (A/H1N1)',
                     'Seasonal Flu', 'Hong Kong flu (1968 pandemic)', 'Spanish flu (1918 pandemic)', 'COVID-19 (Wuhan Coronavirus)']
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

s.store_objects(pandemic_df=pandemic_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
covid19_df = s.load_object('covid19_df')
pandemic_df = pd.merge(left=pandemic_df, right=covid19_df, how='outer', left_index=True, right_index=True, suffixes=('_old', '_new'))
pandemic_df.columns = ['Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU', 'Asymptomatic_transmission',
                       'Year_vaccine_available', 'R0_high', 'R0_low', 'CFR_high', 'CFR_low', 'deaths_per_day_global_rounded',
                       'total_news_mentions_millions_data_retrieved_3rd_mar_2020', 'news_mentions_per_death', 'CFR_new',
                       'R0_new', 'annual_global_fatalities_latest_data_year', 'new_cases_per_year_latest_data_year',
                       'days_of_outbreak', 'fatalty_notes', 'source', 'url']
columns_list = sorted([cn for cn in pandemic_df.columns if ('_low' in cn) or ('_high' in cn) or ('_new' in cn)])
num_regex = re.compile(r'[^0-9.]+')
for disease_name, row_series in pandemic_df[columns_list].iterrows():
    CFR_high = row_series['CFR_high']
    CFR_low = row_series['CFR_low']
    CFR_new = row_series['CFR_new']
    if str(CFR_new) != 'nan':
        CFR_new = float(num_regex.sub('', str(CFR_new)))
    if str(CFR_high) == 'nan':
        pandemic_df.loc[disease_name, 'CFR_high'] = CFR_new
    if str(CFR_low) == 'nan':
        pandemic_df.loc[disease_name, 'CFR_low'] = CFR_new
    if str(CFR_new) != 'nan':
        if CFR_new < CFR_low:
            pandemic_df.loc[disease_name, 'CFR_low'] = CFR_new
        elif CFR_new > CFR_high:
            pandemic_df.loc[disease_name, 'CFR_high'] = CFR_new
    R0_high = row_series['R0_high']
    R0_low = row_series['R0_low']
    R0_new = row_series['R0_new']
    if str(R0_new) != 'nan':
        R0_new = float(num_regex.sub('', str(R0_new)))
    if str(R0_high) == 'nan':
        pandemic_df.loc[disease_name, 'R0_high'] = R0_new
    if str(R0_low) == 'nan':
        pandemic_df.loc[disease_name, 'R0_low'] = R0_new
    if str(R0_new) != 'nan':
        if R0_new < R0_low:
            pandemic_df.loc[disease_name, 'R0_low'] = R0_new
        elif R0_new > R0_high:
            pandemic_df.loc[disease_name, 'R0_high'] = R0_new
columns_list = ['Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU', 'Asymptomatic_transmission',
                'Year_vaccine_available', 'R0_high', 'R0_low', 'CFR_high', 'CFR_low', 'deaths_per_day_global_rounded',
                'total_news_mentions_millions_data_retrieved_3rd_mar_2020', 'news_mentions_per_death',
                'annual_global_fatalities_latest_data_year', 'new_cases_per_year_latest_data_year',
                'days_of_outbreak', 'fatalty_notes', 'source', 'url']
pandemic_df = pandemic_df[columns_list]
pandemic_df.sample(5).T.sample(6).T

In [ ]:

covid19_df = s.load_object('covid19_df')
covid19_df.index = ['Tuberculosis', 'Hepatitis B', 'Pneumonia', 'HIV ', 'Malaria ', 'Shigellosis', 'Rotavirus', 'Seasonal Flu',
                    'Norovirus', 'Whooping Cough', 'Typhoid', 'Cholera', 'Meningitis', 'Measles', 'Rabies ', 'Yellow Fever',
                    'Leishmaniasis', 'Echinococcosis', 'COVID-19 (Wuhan Coronavirus)', 'Dengue Fever', 'Hepatitis A',
                    'Chicken Pox', 'Sleeping Sickness', 'Ebola', 'SARS', 'MERS']
s.store_objects(covid19_df=covid19_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
covid19_df = s.load_object('covid19_df')
check_for_typos(left_list=covid19_df.index.tolist(), right_list=pandemic_df.index.tolist(), verbose=False).sort_values('left_item')

In [ ]:

def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = bs(page_html, 'html.parser')
    
    return page_soup

In [ ]:

tables_url = 'https://www.zorinaq.com/pub/ncov-comparison.html'
ncov_path = os.path.join(s.data_folder, 'html', 'ncov-comparison.html')
#page_soup = get_page_soup(tables_url)
tables_list = get_page_tables(tables_url)

In [ ]:

import numpy as np

pandemic_df = tables_list[0].copy()
pandemic_df.columns = ['Disease', 'Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU',
                       'Asymptomatic_transmission', 'Year_vaccine_available']

def f(x):
    result = re.sub(r' *\(', ' (', str(x))
    
    return result

pandemic_df.Disease = pandemic_df.Disease.map(f)

def f(x):
    result = re.sub(r'⁹', '', str(x))
    
    return result

pandemic_df.Transmission = pandemic_df.Transmission.map(f)
pandemic_df['R0_high'] = np.nan

def f(x):
    result = str(x).split('-')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['R0_high'] = pandemic_df.R0.map(f)
pandemic_df['R0_low'] = np.nan

def f(x):
    result_list = str(x).split('-')
    if len(result_list) < 3:
        result = result_list[0]
    else:
        result = result_list[1].split(' ')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['R0_low'] = pandemic_df.R0.map(f)
pandemic_df['CFR_high'] = np.nan

def f(x):
    result = str(x).split('-')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['CFR_high'] = pandemic_df.Case_fatality_ratio.map(f)
pandemic_df['CFR_low'] = np.nan

def f(x):
    result_list = str(x).split('-')
    result = result_list[0]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['CFR_low'] = pandemic_df.Case_fatality_ratio.map(f)
#print(pandemic_df.R0.unique().tolist())
#print(pandemic_df.R0.map(f).unique().tolist())
s.store_objects(pandemic_df=pandemic_df)
pandemic_df.sample(5).T

In [407]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_States_medical_cases'
tables_list = get_page_tables(tables_url)

[(0, (43, 61)), (1, (30, 56)), (2, (66, 2)), (4, (22, 2)), (5, (21, 2)), (14, (11, 2)), (15, (10, 2)), (17, (8, 2)), (18, (7, 2)), (6, (5, 2)), (7, (5, 2)), (16, (5, 2)), (10, (4, 2)), (8, (3, 2)), (11, (3, 2)), (12, (3, 2)), (13, (2, 2)), (3, (1, 2)), (9, (1, 2))]


In [ ]:

us_deaths_df = tables_list[1].copy()
us_deaths_df.columns = ['Date', 'West_AK', 'West_AZ', 'West_CA', 'West_CO', 'West_HI', 'West_ID', 'West_MT', 'West_NM', 'West_NV',
                        'West_OR', 'West_UT', 'West_WA', 'West_WY', 'Midwest_IA', 'Midwest_IL', 'Midwest_IN', 'Midwest_KS',
                        'Midwest_MI', 'Midwest_MN', 'Midwest_MO', 'Midwest_ND', 'Midwest_NE', 'Midwest_OH', 'Midwest_OK', 'Midwest_SD',
                        'Midwest_WI', 'South_AL', 'South_AR', 'South_FL', 'South_GA', 'South_KY', 'South_LA', 'South_MS', 'South_NC',
                        'South_SC', 'South_TN', 'South_TX', 'South_VA', 'Northeast_CT', 'Northeast_DC', 'Northeast_DE',
                        'Northeast_MA', 'Northeast_MD', 'Northeast_ME', 'Northeast_NH', 'Northeast_NJ', 'Northeast_NY',
                        'Northeast_PA', 'Northeast_RI', 'Northeast_VT', 'Deaths_New', 'Deaths_Cumulative']
columns_list = ['Date', 'Deaths_New', 'Deaths_Cumulative']
us_deaths_df = us_deaths_df[columns_list].loc[:21]
date_format = '%b %d, %Y'
us_deaths_df.Date = us_deaths_df.Date.map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
us_deaths_df.set_index('Date', drop=True, inplace=True)
italy_df = s.load_object('italy_df')
italy_df.set_index('Date', drop=True, inplace=True)
columns_list = ['Deaths_New', 'Deaths_Cumulative']
tracking_df = us_deaths_df[columns_list].merge(italy_df[columns_list], how='outer',
                                               left_index=True, right_index=True, suffixes=('_usa', '_italy'))
for column_name in tracking_df.columns:
    tracking_df[column_name] = pd.to_numeric(tracking_df[column_name], errors='coerce')

In [ ]:

date_format = '%b %d, %Y'
usa_df.Date = usa_df.Date.map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
s.store_objects(usa_df=usa_df)
s.save_dataframes(usa_df=usa_df)

In [ ]:

usa_df = s.load_object('usa_df')
usa_df.set_index('Date', drop=True, inplace=True)
italy_df = s.load_object('italy_df')
italy_df.set_index('Date', drop=True, inplace=True)
columns_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
tracking_df = usa_df[columns_list].merge(italy_df[columns_list], how='outer',
                                         left_index=True, right_index=True, suffixes=('_usa', '_italy'))
s.store_objects(tracking_df=tracking_df)
s.save_dataframes(include_index=True, tracking_df=tracking_df)

In [428]:

china_df = s.load_object('china_df')
china_df.Date = china_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
china_df.set_index('Date', drop=True, inplace=True)
czech_df = s.load_object('czech_df')
czech_df.Date = czech_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
czech_df.set_index('Date', drop=True, inplace=True)
columns_list = list(set(china_df.columns).intersection(set(czech_df.columns)))
merge_df = china_df[columns_list].merge(czech_df[columns_list], how='outer',
                                        left_index=True, right_index=True,
                                        suffixes=('_china', '_czech'))
print('czech_df:', merge_df.index.has_duplicates)


germany_df = s.load_object('germany_df')
germany_df.Date = germany_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
germany_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(germany_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_germany'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany']
print('germany_df:', merge_df.index.has_duplicates)


hong_kong_df = s.load_object('hong_kong_df')
hong_kong_df.Date = hong_kong_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
hong_kong_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(hong_kong_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_hong_kong'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong']
print('hong_kong_df:', merge_df.index.has_duplicates)


iran_df = s.load_object('iran_df')
iran_df.Date = iran_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
iran_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(iran_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_iran'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong', 'Confirmed_New_iran']
print('iran_df:', merge_df.index.has_duplicates)


italy_df = s.load_object('italy_df')
italy_df.Date = italy_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
italy_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(italy_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_italy'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy']
print('italy_df:', merge_df.index.has_duplicates)


japan_df = s.load_object('japan_df')
japan_df.Date = japan_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
japan_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(japan_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_japan'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan']
print('japan_df:', merge_df.index.has_duplicates)


netherlands_df = s.load_object('netherlands_df')
netherlands_df.Date = netherlands_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
netherlands_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(netherlands_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_netherlands'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands']
print('netherlands_df:', merge_df.index.has_duplicates)


singapore_df = s.load_object('singapore_df')
singapore_df.Date = singapore_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
singapore_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(singapore_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_singapore'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands', 'Confirmed_New_singapore']
print('singapore_df:', merge_df.index.has_duplicates)


slovakia_df = s.load_object('slovakia_df')
slovakia_df.Date = slovakia_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
slovakia_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(slovakia_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_slovakia'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia']
print('slovakia_df:', merge_df.index.has_duplicates)


south_korea_df = s.load_object('south_korea_df')
south_korea_df.Date = south_korea_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
south_korea_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(south_korea_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_south_korea'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea']
print('south_korea_df:', merge_df.index.has_duplicates)


spain_df = s.load_object('spain_df')
spain_df.Date = spain_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
spain_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(spain_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_spain'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea', 'Confirmed_New_spain']
print('spain_df:', merge_df.index.has_duplicates)


uk_df = s.load_object('uk_df')
uk_df.Date = uk_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
uk_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(uk_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_uk'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea',
                    'Confirmed_New_spain', 'Confirmed_New_uk']
print('uk_df:', merge_df.index.has_duplicates)


usa_df = s.load_object('usa_df')
usa_df.Date = usa_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
usa_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(usa_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_usa'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong', 'Confirmed_New_iran',
 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands', 'Confirmed_New_singapore',
 'Confirmed_New_slovakia', 'Confirmed_New_south_korea', 'Confirmed_New_spain', 'Confirmed_New_uk', 'Confirmed_New_usa']
print('usa_df:', merge_df.index.has_duplicates)

czech_df: False
germany_df: False
hong_kong_df: False
iran_df: False
italy_df: False
japan_df: False
netherlands_df: False
singapore_df: False
slovakia_df: False
south_korea_df: False
spain_df: False
uk_df: False
usa_df: False
